# Milestone 1
ggroup 5_

_Authors: Vignesh, Dustin, Aidan, Javairia_

## Section 1: Download the Data

In [ ]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

# code adapted from source 4 

article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/"

response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    #print(file)
    if file["name"] in files_to_dl:
        print(file['name'])
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])


with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

data.zip


## Section 2: Combining the Data with Dask 

In [ ]:
import dask.dataframe as dd

In [ ]:
# Select column names
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']

# Get extension for all files
all_files = "../data/*NSW.csv"

# Combine all files
ddf = dd.read_csv(all_files, assume_missing=True, usecols=use_cols, include_path_column=True)

# Create model column
ddf['model'] = ddf['path'].str.split("/", expand=True, n=10)[10].str.split("_", expand=True, n=3)[0]

# Drop path column
ddf.drop(['path'], axis=1)

# Write combined data to single file
ddf.to_csv("../data/combined_NSW.csv", single_file=True)

## Section 3: Loading the combined CSV to memory

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%load_ext rpy2.ipython
%load_ext memory_profiler

In [ ]:
!pip install matplotlib

### Loading in Chunks

In [ ]:
def get_counts(column, file = "../data/combined_NSW.csv"):
    counts = pd.Series(dtype=int)

    for chunk in pd.read_csv(file, chunksize=10_000):
        counts = counts.add(chunk[column].value_counts(), fill_value=0)

    return counts

In [ ]:
chunk_mem = %memit -o get_counts('lat_max')
chunk_time = %timeit -o get_counts('lat_max')

### Dask

In [ ]:
dask_mem = %memit -o dd.read_csv("../data/combined_NSW.csv")['lat_max'].value_counts().compute()
dask_time = %timeit -o dd.read_csv("../data/combined_NSW.csv")['lat_max'].value_counts().compute()

### Loading only columns of interest

In [ ]:
col_subset_mem = %memit -o pd.read_csv("../data/combined_NSW.csv", usecols=['lat_max'])['lat_max'].value_counts()
col_subset_time = %timeit -o pd.read_csv("../data/combined_NSW.csv", usecols=['lat_max'])['lat_max'].value_counts()

### Loading with `low_memory=True`

In [ ]:
low_mem = %memit -o pd.read_csv("../data/combined_NSW.csv", usecols=['lat_max'],low_memory=True)['lat_max'].value_counts()
low_time = %timeit -o pd.read_csv("../data/combined_NSW.csv", usecols=['lat_max'],low_memory=True)['lat_max'].value_counts()

### Comparison

In [ ]:
analysis = pd.DataFrame({
    "Method": ["Chunking", "Dask", "Subsetting Columns", "Low Memory"],
    "Time": [np.mean(chunk_time.all_runs), np.mean(dask_time.all_runs), np.mean(col_subset_time.all_runs), np.mean(low_time.all_runs)],
    "Memory": [chunk_mem.mem_usage[0], dask_mem.mem_usage[0], col_subset_mem.mem_usage[0], low_mem.mem_usage[0]]
})

analysis

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sc = ax.scatter(analysis.Time, analysis.Memory, c = pd.Categorical(analysis.Method).codes, cmap='Dark2')
ax.legend(sc.legend_elements()[0], analysis.Method, title="Method")
plt.show()

### Discussion

From the experiments, The slowest was chunking. This makes sense since we would have to do multiple iterations to get through the entire data. However, the trade off was the amount of memory used: Chunking used the least amount of memory among all methods tested. 

The fastest method was loading only the column of interest. This was faster than using Dask. The use of `low_memory=True` did not affect the memory usage by much. If we were to pick one of these methods, it would be the column sub-setting method. 

In [ ]:
merge = pd.read_csv("../data/combined_NSW.csv", usecols=['model'])

## Section 4: Transfer from Python to R

In [ ]:
import pyarrow.feather as feather
import pyarrow as pa
import pyarrow.parquet as pq
import rpy2_arrow.pyarrow_rarrow as pyra

In [ ]:
%%R

library(tidyverse)
library(here)
library(feather)
library(arrow)

In [ ]:
%%time
%%R -i merge 

# transfer dataframe to R as python dataframe

start_time <- Sys.time()
result <- merge %>% count(model)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
%%time
# write the dataframe to feather format 

feather.write_feather(merge, 'data/final_data.feather')

In [ ]:
%%time

# write the dataframe to arrow and then parquet format
# code adapted from source 1

table = pa.Table.from_pandas(merge)
pq.write_table(table, 'data/final_data.parquet')

In [ ]:
%%time

# write the dataframe to arrow format 
# code adapted from source 4 

final_table = pyra.converter.py2rpy(table)

In [ ]:
%%time
%%R 

# transfer dataframe to R as a feather

file_path = here("data", "final_data.feather")
start_time <- Sys.time()
df <- read_feather(file_path)
result <- df %>% count(model)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
%%time
%%R 

# transfer dataframe to R as a parquet 

file_path = here("data", "final_data.parquet")
start_time <- Sys.time()
df <- read_parquet(file_path)
result <- df %>% count(model)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
%%time
%%R -i final_table

# transfer dataframe to R from arrow 
# code adapted from source 4

start_time <- Sys.time()
result <- final_table %>% collect() %>% count(model)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

In [ ]:
%%sh

du -sh data/final_data.feather
du -sh data/final_data.parquet

**Reasoning**

From the exploration of the different memory and time usage, we can see that parquet store less memory then the feather files as inferred from the shell command above that tells us the directories' usage. From our research, we infer that this is due to the use of dictionary encoding and certain compressions that make this possible (2). Furthermore, we noticed the time it takes to do a basic query is much faster for parquet and feather files than passing the pandas dataframe through pandas exchange because it may be reading all the rows of the file to get the answer to our simple count query. However, parquet files store the metadata of the file and can easily access the columns since they are stored in a columnar format (3) and read the files without having to loop through everything. However, the time difference between feather and parquet files is minimal. We hypothesize that this may be due to the fact that our files are not large enough for us to see the difference. We have also considered using an arrow exchange to go from Python to R. In general, arrow is great for in-memory computing (5) and we noticed it was faster than parquet and feather files. It is also less expensive to write than the parquet file format (5). We have also found that parquet and arrow files are used together as a way of performing many operations in the arrow format and then storing the file as a parquet for long term archival storage (5). Both parquet and arrow files are easily integratable to Spark which will be used a tool for later milestones (2 & 5). But this arrow exchange only has select operations that can be performed in R and this process is still in development as mentioned in our 525 lecture 2 (4). Overall, the conversion to parquet file was easy, simple and the fastest while allowing us to stick to the normal operations in R to read the data into a dataframe again so because of this and the fact that we don't anticipate no additional data that will be added at a later date, we will be using parquet as our final choice.

**References**

1. https://stackoverflow.com/questions/41066582/python-save-pandas-data-frame-to-parquet-file
2. https://stackoverflow.com/questions/48083405/what-are-the-differences-between-feather-and-parquet
3. https://luminousmen.com/post/big-data-file-formats
4. https://github.ubc.ca/MDS-2020-21/DSCI_525_web-cloud-comp_students
5. https://stackoverflow.com/questions/56472727/difference-between-apache-parquet-and-arrow